# Import libraries

In [1]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import json
import pandas as pd

# Get Cache

# Concurrency

# Get Sessions

In [24]:
session = requests.Session()

# Get Urls

In [51]:
def get_page_links(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(url)
    driver.find_element(By.XPATH,'//*[@id="uc-btn-accept-banner"]').click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    s_links = soup.select('li.search-results__item a')

    url_list = []
    for element in s_links:
        link = element["href"]
        if '/agency' in link or '/en/classified/' not in link: 
            pass

        elif 'https://www.immoweb.be' not in link:
            link = 'https://www.immoweb.be' + link
            url_list.append(link)
        else:
            url_list.append(link)
    return url_list


def get_element_url_list():
    all_urls = []
    for i,x in enumerate(range(1,15)):
        urls = get_page_links(f'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page={x}&orderBy=relevance')
        for url in urls:
            start = time.time()
            if url not in all_urls:
                all_urls.append(url)
            end = time.time()
        print('got Url',i,end-start)
    print(len(all_urls))
    return all_urls

#get houses
#get appartsments

In [52]:
url_list = get_element_url_list()

got Url 0 7.152557373046875e-07


KeyboardInterrupt: 

# Get Elements_properties_raw

In [48]:
def get_property(url):
    req_element = session.get(url)
    soup = BeautifulSoup(req_element.text, 'html.parser')
    element_info = soup.select('div.classified script')
    text = element_info[0].text

    return json.loads("".join(("".join(text.split('=')[1:]).split(";")[:-1])))

In [49]:
main_dictionary = dict()
start = time.time()
for index,url in enumerate(url_list):
    start = time.time()
    main_dictionary[url] = get_property(url)
    end = time.time()
    print('scraped',index,' time ',end-start,url)
end = time.time()
print(end-start)

scraped 0  time  0.748204231262207 https://www.immoweb.be/en/classified/house/for-sale/ixelles/1050/10008022?searchId=633703e5d4c02
scraped 1  time  0.4036402702331543 https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=633703e5d4c02
scraped 2  time  0.1976306438446045 https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=633703e5d4c02
scraped 3  time  0.18146371841430664 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/wavre/1300/10143225?searchId=633703e5d4c02
scraped 4  time  0.18513703346252441 https://www.immoweb.be/en/classified/apartment/for-sale/evere/1140/10142884?searchId=633703e5d4c02
scraped 5  time  0.6044204235076904 https://www.immoweb.be/en/classified/house/for-sale/waimes/4950/10143254?searchId=633703e5d4c02
scraped 6  time  0.6380059719085693 https://www.immoweb.be/en/classified/apartment-block/for-sale/ronse/9600/10143136?searchId=633703e5d4c02
scraped 7  time  0.234660

IndexError: list index out of range

In [50]:
for index,url in enumerate(url_list):
    print(index,url)

0 https://www.immoweb.be/en/classified/house/for-sale/ixelles/1050/10008022?searchId=633703e5d4c02
1 https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=633703e5d4c02
2 https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=633703e5d4c02
3 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/wavre/1300/10143225?searchId=633703e5d4c02
4 https://www.immoweb.be/en/classified/apartment/for-sale/evere/1140/10142884?searchId=633703e5d4c02
5 https://www.immoweb.be/en/classified/house/for-sale/waimes/4950/10143254?searchId=633703e5d4c02
6 https://www.immoweb.be/en/classified/apartment-block/for-sale/ronse/9600/10143136?searchId=633703e5d4c02
7 https://www.immoweb.be/en/classified/house/for-sale/woluwe-saint-pierre/1150/10142887?searchId=633703e5d4c02
8 https://www.immoweb.be/en/classified/exceptional-property/for-sale/dalhem/4607/10143366?searchId=633703e5d4c02
9 https://www.immoweb.be/en/classified/h

# Save into json

In [17]:
with open("Main_datas.json","w") as main_data_file:
    main_data_file.write(json.dumps(main_dictionary))

# Get Elements_properties_list

In [18]:
data_main_list = list()
start = time.time()
for index,element in enumerate(main_dictionary.values()):
    element_data = {
    'id': element['id'],
    'url': list(main_dictionary.keys())[index],
    'Locality': element['property']['location']['postalCode'] if element['property']['location'] is not None else None,
    'Type of property': element['property']['type'] if element['property']['type'] is not None else None,
    'Subtype of property': element['property']['subtype'] if element['property']['subtype'] is not None else None,
    'Price': element['price']['mainValue'] if element['price'] is not None else None,
    'Type of sale': element['price']['type'] if element['price'] is not None else None,
    'Number of rooms': element['property']['roomCount'] if element['property']  is not None else None,
    'Area': element['property']['netHabitableSurface'] if element['property'] is not None else None,
    'Fully equipped kitchen': element['property']['kitchen']['type'] if element['property']['kitchen'] is not None else None,
    'Furnished': element['transaction']['sale']['isFurnished'] if element['transaction']['sale'] is not None else None,
    'Open fire': element['property']['fireplaceExists'] if element['property'] is not None else None,
    'Terrace': element['property']['hasTerrace'] if element['property'] is not None else None,
    'Terrace area': element['property']['terraceSurface'] if element['property'] is not None else None,
    'Garden': element['property']['hasGarden'] if element['property'] is not None else None,
    'Garden area': element['property']['gardenSurface'] if element['property'] is not None else None,
    'Surface of the land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Surface area of the plot of land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Number of facades ': element['property']['building']['facadeCount'] if element['property']['building'] is not None else 1,
    'Swimming pool': element['property']['hasSwimmingPool'] if element['property'] is not None else None,
    'State of the building': element['property']['building']['condition'] if element['property']['building'] is not None else None
    }
    data_main_list.append(element_data)
    print('Done',list(main_dictionary.keys())[index])
    
end = time.time()
print(end-start)

Done https://www.immoweb.be/en/classified/house/for-sale/ixelles/1050/10008022?searchId=6336981899ac0
Done https://www.immoweb.be/en/classified/10125449
Done https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=6336981899ac0
Done https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-st-lambert/1200/10052020?searchId=6336981899ac0
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent/9000/10140797?searchId=6336981899ac0
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/roeselare/8800/10140738?searchId=6336981899ac0
Done https://www.immoweb.be/en/classified/house/for-sale/remicourt/4350/10140833?searchId=6336981899ac0
Done https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10140583?searchId=6336981899ac0
Done https://www.immoweb.be/en/classified/apartment/for-sale/deinze/9800/10130388?searchId=6336981899ac0
Done https://www.immoweb.be/en/class

In [7]:
print (data_main_list)

[{'id': 10008022, 'url': 'https://www.immoweb.be/en/classified/house/for-sale/ixelles/1050/10008022?searchId=6336981899ac0', 'Locality': '1050', 'Type of property': 'HOUSE', 'Subtype of property': 'HOUSE', 'Price': 1790000, 'Type of sale': 'residential_sale', 'Number of rooms': 5, 'Area': None, 'Fully equipped kitchen': 'USA_HYPER_EQUIPPED', 'Furnished': False, 'Open fire': False, 'Terrace': True, 'Terrace area': None, 'Garden': True, 'Garden area': 140, 'Surface of the land': 290, 'Surface area of the plot of land': 290, 'Number of facades ': 2, 'Swimming pool': False, 'State of the building': 'AS_NEW'}, {'id': 10125449, 'url': 'https://www.immoweb.be/en/classified/10125449', 'Locality': '1070', 'Type of property': 'APARTMENT_GROUP', 'Subtype of property': 'APARTMENT_GROUP', 'Price': None, 'Type of sale': 'group_sale', 'Number of rooms': None, 'Area': None, 'Fully equipped kitchen': None, 'Furnished': False, 'Open fire': False, 'Terrace': None, 'Terrace area': None, 'Garden': None, 'G

# Save information into CSV

In [21]:
df = pd.DataFrame(data_main_list)
df.to_csv("df_immo.csv", index=False)